# Seleção de atributos

In [1]:
# Manipulação dos dados
import numpy  as np
import scipy  as sp
import pandas as pd

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn           as sns
%matplotlib inline

In [2]:
# Carregando os dados
data = pd.read_excel('../../Dados/Banco labels.xlsx')

In [3]:
data.head(5)

,TB_desfecho_final_Dri,Tbafranio,Obs_desfechoTB_Dri_Dani,Data_inclusão,Nome,Desfecho_tuberculose_lista_Mauro,Desfecho_TB_162_Revisão_ADRIANA,Observaça0_desfecho_TB,Resultado_BAAR_1,Resultado_BAAR_2,...,Perdeu_aula_hoje_CA,Deixou_de_ganhar_dinheiro_hoje_CA,Quanto_deixou_ganhar_CA,Caso_acompanhado_quem_custeou_tudo_CA,Valor_CA,Avaliador_custo_CA,Data_avaliação_custo_CA,Resultado_cultura_final < 3 & Realizou_RX = 1 (FILTER),ANO de inclusao,Novocriteriotbafranio
0,TB,SIM,NaN,2008-04-03,NaN,9,TB,TB PROBABILIDADE,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2008,1
1,TB,SIM,NaN,2006-10-03,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2006,1
2,TB,SIM,NaN,2007-06-21,NaN,TB,TB,TB CONFIRMADA,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2007,1
3,TB,SIM,NaN,2009-09-30,NaN,TB,TB,NaN,Negativo,Negativo,...,IGN,IGN,Ignorado,IGN,ignorado,9,1999-09-09,Selected,2009,1
4,TB,SIM,NaN,2009-06-19,NaN,TB,NaN,NaN,Negativo,Negativo,...,Não,Não,"""Não se aplica""",não se aplica,não se aplica,Marcia dos Santos,2009-06-19,Not Selected,2009,1


## 1. Pré-processamento

In [4]:
# Renomeando o desfecho
data.rename(index=str,columns={'TB_desfecho_final_Dri' : 'desfecho'},inplace=True)

In [21]:
dados = data.copy()

In [36]:
# Features pré-selecionadas
features = ['desfecho', 'TTO_anterior_TB_triagem_enfermeiro', 'Cicatriz_BCG',
       'Idade', 'Sexo', 'Raça', 'Estado_civil', 'Tem_companheiro', 'Tosse',
       'Semanas_tosse', 'Expectoração', 'Hemoptóicos',
       'Hemoptise', 
       'Sudorese_noturna', 'Febre',
       'Dispnéia', 'Perda_de_apetite',
       'Perda_peso_10percent', 
       'Dor_torácica', 'Internação_hospital_2anos',
       'Prisão_2anos', 'Delegacia_2anos', 'Contato_TBP_2anos', 'Fuma',
       'CAGE']

# Filtrando as features
dados = dados[features]

# Alterando labels
dados.replace(['TB', 'Não TB', 'IGN'],['TB+','TB-','ignorado'],inplace=True)

# Filtrando para desfechos da doença desconhecidos
dados = dados[(dados.desfecho == 'TB+') | (dados.desfecho == 'TB-')]

# Pré-processamento dos dados básico
dados.Idade.replace(999,45.0,inplace=True)
dados.replace([np.nan],['ignorado'],inplace=True)
dados.replace(['não se aplica'],[0],inplace=True)
dados.Dispnéia.replace([2],['ignorado'],inplace=True)
dados.Sexo.replace([2],['ignorado'],inplace=True)
dados.Sudorese_noturna.replace([9],['ignorado'],inplace=True)
dados.Estado_civil.replace([0],['ignorado'],inplace=True)
dados.Raça.replace([8],['ignorado'],inplace=True)
dados.Semanas_tosse.replace(['não se aplica','crônico+sempre'],[0,5],inplace=True)

In [23]:
# Tosse maior que 3 semanas
idx = dados[dados.Semanas_tosse != 'ignorado'].index
dados.Semanas_tosse[idx] = dados[dados.Semanas_tosse != 'ignorado'].Semanas_tosse.apply(lambda x: 1 if x > 3 else 0)
dados.Semanas_tosse.replace([0,1],['Não','Sim'],inplace=True)

/Users/Michel/anaconda/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## 2. Construção dos datasets


Duas abordagens:

- Abordagem 1: Utilizar somente a codificação one-hot encoding em Fuma e Estado_civil
- Abordagem 2: Considerar todas as variáveis como categóricas

In [31]:
# Pré-processamento final: ONE-HOT encoding parcial e total
features = list(dados.columns)[1:]
features_ohe = ['Estado_civil','Fuma']

In [32]:
# Dataframe com dummies parciais
dados_abordagem_1 = dados.copy()

for f in features_ohe:
    col = pd.get_dummies(dados_abordagem_1[f],prefix=f)
    dados_abordagem_1.drop([f],axis=1,inplace=True)
    dados_abordagem_1 = dados_abordagem_1.join(col)

In [34]:
# Dataframe com dummies totais
dados_abordagem_2 = dados.copy()

for f in features:
    if f == 'Idade':
        continue
    col = pd.get_dummies(dados_abordagem_2[f],prefix=f)
    dados_abordagem_2.drop([f],axis=1,inplace=True)
    dados_abordagem_2 = dados_abordagem_2.join(col)

In [38]:
# Armazenando as features pré-selecionadas em um arquivo csv
dados_abordagem_1.to_csv('../../Dados/abordagem1.csv',index=False,columns=list(dados_abordagem_1.columns))
dados_abordagem_2.to_csv('../../Dados/abordagem2.csv',index=False,columns=list(dados_abordagem_2.columns))